# T-Test (Student's t-test)

In [59]:
import numpy as np
import pandas as pd
from scipy import stats

In [14]:
df_iris = pd.read_csv('./datasets/iris.csv')

print(df_iris.head(), '\n')
print(df_iris.describe(), '\n')
print(df_iris['Species'].unique())

   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa 

               Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
count  150.000000     150.000000    150.000000     150.000000    150.000000
mean    75.500000       5.843333      3.054000       3.758667      1.198667
std     43.445368       0.828066      0.433594       1.764420      0.763161
min      1.000000       4.300000      2.000000       1.000000      0.100000
25%     38.250000       5.100000      2.800000       1.600000      0.300000
50%     75.500000       5.800000      3.000000       4.350000      1.300000
75% 

## 1. 單一樣本 T 檢定

In [48]:
# 從資料中隨機抽取 20 筆 Iris-setosa 資料做檢定
s_iris_setosa = np.random.choice(df_iris[df_iris['Species'] == 'Iris-setosa']['SepalLengthCm'], 20)

# 由於資料為小樣本, 需進行常態性檢定 (Shapiro-Wilk test)
'''
Shapiro-Wilk test is a test of normality, it determines whether the given sample comes from the normal distribution or not. 
Shapiro-Wilk’s test or Shapiro test is a normality test in frequentist statistics. 
Ho: Sample is from the normal distributions.(Po > 0.05)
Ha: Sample is not from the normal distributions.
'''
print(stats.shapiro(s_iris_setosa), '\n')
print(f'P-Value = {stats.shapiro(s_iris_setosa)[1]} > 0.05, do not reject Ho, sample is from the normal distributions.')

ShapiroResult(statistic=0.9466801881790161, pvalue=0.31948909163475037) 

P-Value = 0.31948909163475037 > 0.05, do not reject Ho, sample is from the normal distributions.


In [66]:
'''
H0: The mean of speal length of Iris-setosa is not greater than 5
H1: The mean of speal length of Iris-setosa is greater than 5 (one-tailed test)

sigificance level = 0.05, if p-value < 0.05, we reject H0.
It means that the mean of speal length of Iris-setosa is greater than 5
'''

mu_0 = 5

sigificance_level = 0.05

mu = np.mean(s_iris_setosa) # sample mean
df = len(s_iris_setosa) - 1 # degree of freedom
sigma = np.std(s_iris_setosa, ddof=1) # sample standard deviation
se = sigma / np.sqrt(len(s_iris_setosa)) # standard error

t_value = (mu - mu_0) / se # test statistic
print(f'Test statistic = {t_value}')

''' Critical value approach, if t_value > critical value, we reject H0 '''
critical_value = stats.t.ppf(1-sigificance_level, df) # critical value, one-tailed test (right)
print(f'Critical value = {rejection_region}', '\n')

if t_value > critical_value:
    print('Test statistic is in rejection region (t_value > critical value)), reject H0') 
else:
    print('Test statistic is not in rejection region (t_value < critical value), do not reject H0')

Test statistic = 2.527152305619065
Critical value = 1.729132811521367 

Test statistic is in rejection region (t_value > critical value)), reject H0


In [83]:
''' P-value approach, if p-value < sigificance_level, we reject H0 '''
p_value = 1-(stats.t.cdf(t_value, df))
# p_value = stats.t.sf(np.abs(t_value), df)  # 1 - cdf(t_value, df) = sf(t_value, df)

# 也可使用 scipy.stats.ttest_1samp() 快速計算檢定統計量與 P-Value
result = stats.ttest_1samp(s_iris_setosa, mu_0, alternative='greater') # t-test
print(result, '\n')

if p_value < sigificance_level:
    print(f'P-value = {p_value} < {sigificance_level}, reject H0')
else:
    print(f'P-value = {p_value} > {sigificance_level}, do not reject H0')

Ttest_1sampResult(statistic=2.527152305619065, pvalue=0.01026446982031448) 

P-value = 0.010264469820314481 < 0.05, reject H0


In [77]:
''' Confidence interval approach, if mu_0 is not in the confidence interval, we reject H0 '''
confidence_interval = stats.t.interval(1-sigificance_level, df, loc=mu, scale=se)
print(f'Confidence interval = {confidence_interval}', '\n')

if mu_0 < confidence_interval[0] or mu_0 > confidence_interval[1]:
    print(f'mu_0 = {mu_0} is not in the confidence interval, reject H0')
else:
    print(f'mu_0 = {mu_0} is in the confidence interval, do not reject H0')

Confidence interval = (5.026626760400652, 5.2833732395993485) 

mu_0 = 5 is not in the confidence interval, reject H0


模擬實驗計算 P-Value

In [94]:
num_tests = 10000

mu_0 = 5
size = len(s_iris_setosa)
sigma = np.std(s_iris_setosa, ddof=1)

test_statistic = (mu - mu_0) / se

t_value_array = np.zeros(num_tests)
np.random.seed(1)
norm_dist = stats.norm(loc=mu_0, scale=sigma)

for i in range(num_tests):
    sample = norm_dist.rvs(size=size)
    sample_mean = np.mean(sample)
    sample_std = np.std(sample, ddof=1)
    sample_se = sample_std / np.sqrt(size)
    t_value_array[i] = (sample_mean - mu_0) / sample_se

# 計算 array 中大於檢定統計量的佔比, 即為 p-value (雙尾需乘以 2)
sum(t_value_array > test_statistic) / num_tests
# 超過比例越小, 代表 p-value 越小, 樣本觀察值越不可能來自於 H0 的母體分配, 傾向於拒絕 H0

0.0123

## 2. 成對樣本 T 檢定

### 2-1. 兩相依母體 T 檢定: 同一個樣本在不同情況的差異
檢定學生在第二次考試成績有沒有進步，需要計算兩次考試成績的差異，再進行 T 檢定。與單一樣本 T 檢定作法相同：

In [14]:
diff = df_grade['Difference']
'''
H0: The mean of difference between Exam 1 and Exam2 is qual to 0
Ha: The mean of difference between Exam 1 and Exam2 is not qual to 0 (two-tailed test) 
'''
mu_0 = 0
alpha = 0.05

result = stats.ttest_1samp(diff, mu_0)
print(result, '\n')

def p_value_approach(p_value, sigificance_level):
    if p_value < sigificance_level:
        print(f'P-value = {p_value} < {sigificance_level}, reject H0')
    else:
        print(f'P-value = {p_value} > {sigificance_level}, do not reject H0')

p_value_approach(result[1], alpha)

Ttest_1sampResult(statistic=0.7497768853141169, pvalue=0.4649871003972206) 

P-value = 0.4649871003972206 > 0.05, do not reject H0


### 2-2. 獨立樣本 T 檢定
以 iris.csv 為例，想檢定 Iris-setosa 與 Iris-virginica 的 sepal length 是否有顯著差異，需要先將資料分成兩組，再進行 T 檢定。
- 在母體變異數 sigma_1 與 sigma_2 未知情況下，應先進行模型診斷，判斷兩母體變異數是否相等(F 檢定, 滿足同質變異數假設)
- 若兩母體變異數不相等，則應使用 Welch's t-test

In [38]:
df_iris = pd.read_csv('iris.csv')
print(df_iris['Species'].unique())

sepal_setosa = np.random.choice(df_iris[df_iris['Species'] == 'Iris-setosa']['SepalLengthCm'], 20)
sepal_virginica = np.random.choice(df_iris[df_iris['Species'] == 'Iris-virginica']['SepalLengthCm'], 20)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [56]:
'''
H0: mu_s - mu_v = 0
Ha: mu_s - mu_v != 0 (two-tailed test)
'''

const = 0

mean_setosa = sepal_setosa.mean()
mean_virginica = sepal_virginica.mean()

sigma_setosa = sepal_setosa.std(ddof=1)
sigma_virginica = sepal_virginica.std(ddof=1)

n_setosa = len(sepal_setosa)
n_virginica = len(sepal_virginica)

# 計算 Welch 自由度
a = ((sigma_setosa ** 2) / n_setosa)
b =  ((sigma_virginica ** 2) / n_virginica)
df_welch = (a + b) ** 2 / ((a ** 2) / (n_setosa - 1) + (b ** 2) / (n_virginica - 1))
print(f'Degree of Freedom = {df_welch}', '\n')

t_value = (mean_setosa - mean_virginica - const) / (np.sqrt(sigma_setosa ** 2 / n_setosa + sigma_virginica ** 2 / n_virginica))
print(f'Test Statistics = {t_value}', '\n')

# 計算 p-value
print(f'P-value = {stats.t.sf(np.abs(t_value), df_welch) * 2}')

Degree of Freedom = 26.35427339199482 

Test Statistics = -8.265141252471022 

P-value = 8.602600995826035e-09


In [58]:
# equal_var=False, 代表使用變異數不同, 會進行 Welch's t-test
result = stats.ttest_ind(sepal_setosa, sepal_virginica, equal_var=False)
print(result, '\n')

p_value_approach(result[1], alpha)

Ttest_indResult(statistic=-8.265141252471022, pvalue=8.602600995826035e-09) 

P-value = 8.602600995826035e-09 < 0.05, reject H0
